# REINFORCE with baseline for continuous action space

This is an implementation of REINFORCE with baseline with modifications to deal with bounded continuous action spaces. See Sutton & Barto, Section 13.7.

We make the following changes to the previous notebook:

* Separate network for policy and value function
  
* Adjust policy network to output means and standard deviations of a normal distribution

* Use of truncated normal distribution for bounded action space

* Use of parameters given in the PPO paper, don't do hyper-parameter search anymore

* Set up training and evaluation on the Mujoco environments used in the PPO paper, to be used as a comparison for our PPO implementation

In [1]:
import gymnasium as gym
from gymnasium.wrappers import RescaleAction

import numpy as np

import torch
from torch import nn

from torchrl.modules import TruncatedNormal

In [2]:
# The policy needs to be modified to return a mean tensor and a standard deviations tensor
# - use logits as mean
# - standard deviations separate
# - no dependencies between dimensions of multi-variate normal distribution
class Policy(nn.Module):
    def __init__(self, n_in, n_hidden1, n_hidden2, n_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_in, n_hidden1),
            nn.Tanh(),
            nn.Linear(n_hidden1, n_hidden2),
            nn.Tanh(),
            nn.Linear(n_hidden2, n_out),
        )
        # self.sm = nn.Softmax(dim=0) # probabilities to choose an action
        # self.lsm = nn.LogSoftmax(dim=0) # for log probabilities used in the gradient for REINFORCE

        # From PPO, page 6: 
        # "To represent the policy, we used a fully-connected MLP with two hidden layers of 64 units,
        # and tanh nonlinearities, outputting the mean of a Gaussian distribution, with variable standard
        # deviations, following [Sch+15b; Dua+16]."
        # (See [Sch+15b] (Trust Region Policy Optimization) on page 15.)
        # Initialization of the standard deviations is not fully clear here, 
        # but to initialize them to 1 seems a reasonable first guess.
        self.log_stddevs = nn.Parameter(torch.zeros(n_out))
    
    def forward(self, x): 
        logits = self.net(x)
        # probs = self.sm(logits)
        # scores = self.lsm(logits)
        # return probs, scores, logits # we include the logits in the output for use in the state-value function
        stddevs = torch.exp(self.log_stddevs)
        return logits, stddevs
    
class Value(nn.Module):
    def __init__(self, n_in, n_hidden1, n_hidden2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_in, n_hidden1),
            nn.Tanh(),
            nn.Linear(n_hidden1, n_hidden2),
            nn.Tanh(),
            nn.Linear(n_hidden2, 1), # same as Policy, but output is only one number
        )
    
    def forward(self, x): 
        logit = self.net(x)
        return logit

In [3]:
envs = [
    # "Pendulum-v1",
    # "MountainCarContinuous-v0",
    # "BipedalWalker-v3",
    # "LunarLander-v3",
    # "CarRacing-v3", # image input
    "InvertedPendulum-v5", # OK with gamma = 0.9995, alpha_value = 1e-3, alpha = 1e-3, 1000 episodes
    "InvertedDoublePendulum-v5",
    "Reacher-v5",
    "HalfCheetah-v5",
    "Hopper-v5",
    "Walker2d-v5",
    "Swimmer-v5"
]
from gymnasium.wrappers import RescaleAction
for env_name in envs:
    env = gym.make(env_name, continuous=True) if env_name in ["LunarLander-v3", "CarRacing-v3"] else gym.make(env_name)
    wrapped_env = RescaleAction(env, min_action=-1, max_action=1)
    wrapped_env.reset()
    print("\nEnv name: {}, \n- Observation space: {} \n- Action space:      {} \n- Reward threshold: {}".format(env_name, wrapped_env.observation_space, wrapped_env.action_space, env.spec.reward_threshold))
    # The observation spaces are all from -inf to +inf (with various dimensions)

# TODO
# try to bring rewards for all environments on a similar scale
# same with observations & actions, only difference should be the dimension


Env name: InvertedPendulum-v5, 
- Observation space: Box(-inf, inf, (4,), float64) 
- Action space:      Box(-1.0, 1.0, (1,), float32) 
- Reward threshold: 950.0

Env name: InvertedDoublePendulum-v5, 
- Observation space: Box(-inf, inf, (9,), float64) 
- Action space:      Box(-1.0, 1.0, (1,), float32) 
- Reward threshold: 9100.0

Env name: Reacher-v5, 
- Observation space: Box(-inf, inf, (10,), float64) 
- Action space:      Box(-1.0, 1.0, (2,), float32) 
- Reward threshold: -3.75

Env name: HalfCheetah-v5, 
- Observation space: Box(-inf, inf, (17,), float64) 
- Action space:      Box(-1.0, 1.0, (6,), float32) 
- Reward threshold: 4800.0

Env name: Hopper-v5, 
- Observation space: Box(-inf, inf, (11,), float64) 
- Action space:      Box(-1.0, 1.0, (3,), float32) 
- Reward threshold: 3800.0

Env name: Walker2d-v5, 
- Observation space: Box(-inf, inf, (17,), float64) 
- Action space:      Box(-1.0, 1.0, (6,), float32) 
- Reward threshold: None

Env name: Swimmer-v5, 
- Observation spac

In [4]:
seed = 42
env_name = "InvertedDoublePendulum-v5"

n_hidden1 = 64 
n_hidden2 = 64
gamma = 0.99

# Sutton & Barto use a separate learning rate to update the state-value function parameters
alpha_value = 3e-4
alpha = 3e-4

n_episodes = 20_000
print_every_n_episodes = 500

early_stopping = False

In [5]:
np.random.seed(seed)
torch.manual_seed(seed)

# Initialise the environment
env = gym.make(env_name, continuous=True) if env_name in ["LunarLander-v3", "CarRacing-v3"] else gym.make(env_name)
wrapped_env = RescaleAction(env, min_action=-1, max_action=1)

# Reset the environment to generate the first observation
observation, info = env.reset(seed=seed)
observation = torch.tensor(observation, requires_grad=False, dtype=torch.float32)

policy = Policy(n_in=env.observation_space.shape[0], n_hidden1=n_hidden1, n_hidden2=n_hidden2, n_out=env.action_space.shape[0])

# The state-value function maps states to a single number. From PPO: "We don’t share parameters between the policy and value function [...]". 
value = Value(n_in=env.observation_space.shape[0], n_hidden1=n_hidden1, n_hidden2=n_hidden2)

# Sutton & Barto use separate gradient update steps with separate learning rates for the policy and the state-value function
# Adam with default parameters instead of SGD
optimizer = torch.optim.Adam(policy.parameters(), lr=alpha, maximize=True)
optimizer_value = torch.optim.Adam(value.parameters(), lr=alpha_value, maximize=True)


all_episode_rewards = []

for i_episode in range(1, n_episodes+1):
    
    rewards = [] # T rewards from 1 to T
    observations = [observation] # T observations from 0 to T-1, the policy expects tensors as input
    actions = [] # T actions from 0 to T-1

    # roll-out of one episode following the policy
    done = False
    while not done:
        
        # probabilities for actions
        # Change 1: The policy network outputs means and standard deviations
        pred_means, pred_stddevs = policy(observation)
        
        # sample an action according to the probabilities
        # Change 2: We use the truncated normal distribution on the continuous action space, not the categorical distribution for a discrete set of actions
        TN = TruncatedNormal(loc=pred_means,
                             scale=pred_stddevs,
                             low=env.action_space.low,
                             high=env.action_space.high,
                             tanh_loc=False)
        action = TN.sample()

        # step (transition) through the environment with the action
        # receiving the next observation, reward and if the episode has terminated or truncated
        observation, reward, terminated, truncated, info = env.step(action.detach().numpy())
        observation = torch.tensor(observation, requires_grad=False, dtype=torch.float32)
        done = terminated or truncated

        # build up one episode
        rewards.append(reward)
        observations.append(observation)
        actions.append(action)

        # If the episode has ended then we can reset to start a new episode
        if done:
            observation, info = env.reset()
            all_episode_rewards.append(sum(rewards)) # track total reward for all episodes

    
    # policy updates using policy gradients along each step of the episode
    # We accumulate gradients over one episode and make only one gradient step per episode
    pseudo_losses = []
    value_losses = []
    for t in range(len(rewards)):
        
        # observation at step t during episode, and action taken
        observation = observations[t]
        action = actions[t] 
        
        # discounted return starting from step t
        G = sum(gamma**i * r for i, r in enumerate(rewards[t:]))
        
        # removing the baseline
        value_logit = value(observation)
        delta = G - value_logit
        
        # note that delta does not only depend on the rewards (through G), but also on the parameters of the state-value network (through the value logits)
        # note however, that we don't take the gradient of them
        delta = delta.detach()

        # this is the distribution at the respective time-step
        pred_means, pred_stddevs = policy(observation)
        TN = TruncatedNormal(loc=pred_means,
                             scale=pred_stddevs,
                             low=env.action_space.low,
                             high=env.action_space.high,
                             tanh_loc=False)

        # we want the updated probability of the action that was actually taken
        # Old: use score output of policy network
        # Change 1: now use log_prob of the distribution class
        log_prob = TN.log_prob(action).sum() # If the action space has more than one action, the log probs sum if we assume the actions to be independent.

        # We accumulate gradients over one episode and make only one gradient step per episode
        pseudo_losses.append(gamma**t * delta * log_prob)
        value_losses.append(delta * value_logit)
    
    # We accumulate gradients over one episode and make only one gradient step per episode
    pseudo_loss = torch.stack(pseudo_losses).mean()
    value_pseudo_loss = torch.stack(value_losses).mean()
    
    # We now include a gradient step (also a maximization) of the value function
    optimizer_value.zero_grad()
    value_pseudo_loss.backward()
    optimizer_value.step()

    optimizer.zero_grad()
    pseudo_loss.backward()
    optimizer.step()
    
    # print some statistics every other episode
    if i_episode % print_every_n_episodes == 0:
        print("\nEpisode:", i_episode, "of", n_episodes)
        print("Alpha:", alpha, "Alpha_value:", alpha_value, "Gamma", gamma)
        print("Total reward in this episode:", sum(rewards))
        print("State-value function for initial observation in this episode:", value(observations[0]).item())
        if len(all_episode_rewards) > 100: print("Average reward last 100 episodes:", sum(all_episode_rewards[-100:])/100)

    # TODO early stopping needs lower average reward bound applicable for several environments. Reward scaling and max_timesteps -> gives lower bound for reward per episode?
    # if early_stopping and i_episode > 4000 and np.mean(all_episode_rewards[-200:]) < -200:
    #     print("Run terminated due to early stopping at episode", i_episode)
    #     break

# - When a policy solved the environment for one episode:
    if all_episode_rewards[-1] > env.spec.reward_threshold:
        print("Environment was solved for one episode at episode", i_episode)
        
env.close()


Episode: 500 of 20000
Alpha: 0.0003 Alpha_value: 0.0003 Gamma 0.99
Total reward in this episode: 118.83357804680574
State-value function for initial observation in this episode: 14.551530838012695
Average reward last 100 episodes: 78.24712491262116

Episode: 1000 of 20000
Alpha: 0.0003 Alpha_value: 0.0003 Gamma 0.99
Total reward in this episode: 137.99324974676503
State-value function for initial observation in this episode: 23.07192039489746
Average reward last 100 episodes: 77.95293582569428

Episode: 1500 of 20000
Alpha: 0.0003 Alpha_value: 0.0003 Gamma 0.99
Total reward in this episode: 129.0499625532247
State-value function for initial observation in this episode: 29.62936019897461
Average reward last 100 episodes: 85.2344724982539

Episode: 2000 of 20000
Alpha: 0.0003 Alpha_value: 0.0003 Gamma 0.99
Total reward in this episode: 135.85128145854048
State-value function for initial observation in this episode: 35.12007141113281
Average reward last 100 episodes: 91.02658630441452

E

### Save and load trained policies

In [6]:
import pickle

# Uncomment to save a trained policy:
# pickle.dump(policy, open('policies/REINFORCE_with_baseline_InvertedDoublePendulum_visualize.pkl', 'wb'))

# Uncomment to load a saved policy:
policy = pickle.load(open('policies/REINFORCE_with_baseline_InvertedDoublePendulum_visualize.pkl', 'rb'))

### Plotting training progress

In [7]:
import matplotlib.pyplot as plt
smoothing_interval = 100 # Change for different levels of smoothing
a = np.array(all_episode_rewards)
smoothed_rewards = a[(len(a)%smoothing_interval):].reshape(-1, smoothing_interval).mean(1)

plt.plot(smoothed_rewards)
xtcks = np.arange(0, len(smoothed_rewards), smoothing_interval)
plt.xticks(xtcks,  smoothing_interval*xtcks)

np.where(a > env.spec.reward_threshold) # training steps at which the policy "solved" the environment


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/adrian/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/adrian/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/home/adrian/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start
    self.io_loop.start()
  File "/home/adrian

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

## Visualize the policy


In [ ]:
# Initialise the environment
# TODO this currently does not work for me
env = gym.make(env_name, render_mode="human")


# Reset the environment to generate the first observation
observation, info = env.reset(seed=42)

done = False
while not done:

    pred_means, pred_stddevs = policy(torch.tensor(observation, requires_grad=False, dtype=torch.float32))
    action = pred_means # TODO: should sampling also be stochastic?

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action.detach().numpy())
    
    # If the episode has ended then we can reset to start a new episode
    done = terminated or truncated
    if done:
        observation, info = env.reset()

env.close()

In [ ]:
# TODO one more approach: rendering an image after each step and collecting them to a video
# https://github.com/google-deepmind/dm_control/issues/39
from dm_control import suite
import numpy as np
import cv2

def grabFrame(env):
    # Get RGB rendering of env
    rgbArr = env.physics.render(480, 600, camera_id=0)
    # Convert to BGR for use with OpenCV
    return cv2.cvtColor(rgbArr, cv2.COLOR_BGR2RGB)

# Load task:
env = suite.load(domain_name="cartpole", task_name="swingup")

# Setup video writer - mp4 at 30 fps
video_name = 'video.mp4'
frame = grabFrame(env)
height, width, layers = frame.shape
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (width, height))

# First pass - Step through an episode and capture each frame
action_spec = env.action_spec()
time_step = env.reset()
while not time_step.last():
    action = np.random.uniform(action_spec.minimum,
                               action_spec.maximum,
                               size=action_spec.shape)
    time_step = env.step(action)
    frame = grabFrame(env)
    # Render env output to video
    video.write(grabFrame(env))

# End render to video file
video.release()

# Second pass - Playback
cap = cv2.VideoCapture(video_name)
while(cap.isOpened()):
    ret, frame = cap.read()
    cv2.imshow('Playback', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()

# Exit
cv2.destroyAllWindows()